In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Код разделен на четыре основных этапа:
    
    
1) Работа с данными

2) Обучение классификатора сентимента (настроения)

3) Обучение классификатора рейтинга

4) Оценка работы

Работа с данными

Сначала извлечем все данные из текстовых файлов и оформим датасеты

Сделаем датасет из всех позитивных комментариев для обучения:

In [1]:
import os
import pandas as pd
from tqdm import tqdm

directory_path = '/kaggle/input/avaliacoes-de-filmes-dataset-imdb/aclImdb/train/pos'

data = []

for filename in tqdm(os.listdir(directory_path), desc="Обработка", unit="file"):
    if filename.endswith('.txt'):
        # достаем ID и рейтинг из названия файла
        id_str, rating_str = filename[:-4].split('_')
        review_id = int(id_str)
        rating = int(rating_str)

        file_path = os.path.join(directory_path, filename)

        # достанем текст
        with open(file_path, 'r', encoding='utf-8') as file:
            review_text = file.read().strip()

        data.append({
            'ID': review_id,
            'rating': rating,
            'text': review_text,
            'label': 'pos'
        })

# создадим датафрейм со всеми позитивными тренировочными данными
reviews_df = pd.DataFrame(data)
print(reviews_df.head())


Обработка: 100%|██████████| 12500/12500 [01:51<00:00, 112.37file/s]

     ID  rating                                               text label
0  2714      10  This was one of those wonderful rare moments i...   pos
1   589      10  Have you seen The Graduate? It was hailed as t...   pos
2  2211       8  I don't watch a lot of TV, except for The Offi...   pos
3  2658      10  Kubrick again puts on display his stunning abi...   pos
4  8929       8  First of all, I liked very much the central id...   pos


То же самое для негатива:

In [2]:
import os
import pandas as pd
from tqdm import tqdm

directory_path = '/kaggle/input/avaliacoes-de-filmes-dataset-imdb/aclImdb/train/neg'

data = []

for filename in tqdm(os.listdir(directory_path), desc="Обработка", unit="file"):
    if filename.endswith('.txt'):
        id_str, rating_str = filename[:-4].split('_')
        review_id = int(id_str)
        rating = int(rating_str)

        file_path = os.path.join(directory_path, filename)

        with open(file_path, 'r', encoding='utf-8') as file:
            review_text = file.read().strip()

        data.append({
            'ID': review_id,
            'rating': rating,
            'text': review_text,
            'label': 'neg' 
        })

neg_reviews_df = pd.DataFrame(data)
print(neg_reviews_df.head())


Обработка: 100%|██████████| 12500/12500 [01:42<00:00, 121.55file/s]

     ID  rating                                               text label
0  3606       2  This film is the worst film, but it ranks very...   neg
1  1074       4  I should never have started this film, and sto...   neg
2  4743       1  I'm here again in your local shopping mall (of...   neg
3  7628       1  Black and White film. Good photography. Believ...   neg
4  6812       1  from the start of this movie you soon become a...   neg


Теперь объединим датафреймы в один тренировочный датафрейм:

In [3]:
import pandas as pd

combined_df = pd.concat([reviews_df, neg_reviews_df], axis=0, ignore_index=True)

print(combined_df.head())
print(f"Шейп объединенного тренировочного датасета: {combined_df.shape}")


     ID  rating                                               text label
0  2714      10  This was one of those wonderful rare moments i...   pos
1   589      10  Have you seen The Graduate? It was hailed as t...   pos
2  2211       8  I don't watch a lot of TV, except for The Offi...   pos
3  2658      10  Kubrick again puts on display his stunning abi...   pos
4  8929       8  First of all, I liked very much the central id...   pos
Шейп объединенного тренировочного датасета: (25000, 4)


Теперь проверим датасет на дубликаты

In [4]:
import pandas as pd

combined_df['id'] = combined_df.index
duplicates = combined_df[combined_df.duplicated(subset='text', keep=False)]
sorted_duplicates = duplicates.sort_values(by='text')

print(sorted_duplicates)


         ID  rating                                               text label  \
23479  4102       4  'Dead Letter Office' is a low-budget film abou...   neg   
14197   985       4  'Dead Letter Office' is a low-budget film abou...   neg   
6663   9319       8  .......Playing Kaddiddlehopper, Col San Fernan...   pos   
2441   6069       8  .......Playing Kaddiddlehopper, Col San Fernan...   pos   
19911  7287       2  <br /><br />Back in his youth, the old man had...   neg   
...     ...     ...                                                ...   ...   
24501  6818       3  in this movie, joe pesci slams dunks a basketb...   neg   
8660   8657       9  it's amazing that so many people that i know h...   pos   
254    8654       9  it's amazing that so many people that i know h...   pos   
18898  5085       1  this movie begins with an ordinary funeral... ...   neg   
23148  6642       1  this movie begins with an ordinary funeral... ...   neg   

          id  
23479  23479  
14197  14

Дропнем дубликаты

In [5]:
df_unique = combined_df.drop_duplicates(subset='text')
print(df_unique)

          ID  rating                                               text label  \
0       2714      10  This was one of those wonderful rare moments i...   pos   
1        589      10  Have you seen The Graduate? It was hailed as t...   pos   
2       2211       8  I don't watch a lot of TV, except for The Offi...   pos   
3       2658      10  Kubrick again puts on display his stunning abi...   pos   
4       8929       8  First of all, I liked very much the central id...   pos   
...      ...     ...                                                ...   ...   
24995   1072       4  The first hour of the movie was boring as hell...   neg   
24996  11693       4  A fun concept, but poorly executed. Except for...   neg   
24997   1550       1  I honestly don't understand how tripe like thi...   neg   
24998  12186       1  This remake of the 1962 orginal film'o the boo...   neg   
24999    501       2  La Sanguisuga Conduce la Danza, or The Bloodsu...   neg   

          id  
0          0

Сохраним, чтобы не потерять

In [6]:
df_unique.to_csv('/kaggle/working/combined_train.csv', encoding='utf-8')

Теперь проделаем все то же саме для тестового датасета

In [7]:
import os
import pandas as pd
from tqdm import tqdm

directory_path = '/kaggle/input/avaliacoes-de-filmes-dataset-imdb/aclImdb/test/pos'

data = []

for filename in tqdm(os.listdir(directory_path), desc="Обработка", unit="file"):
    if filename.endswith('.txt'):
        # достаем ID и рейтинг из названия файла
        id_str, rating_str = filename[:-4].split('_')
        review_id = int(id_str)
        rating = int(rating_str)

        file_path = os.path.join(directory_path, filename)

        # достанем текст
        with open(file_path, 'r', encoding='utf-8') as file:
            review_text = file.read().strip()

        data.append({
            'ID': review_id,
            'rating': rating,
            'text': review_text,
            'label': 'pos'
        })

# создадим датафрейм со всеми позитивными тренировочными данными
reviews_df = pd.DataFrame(data)
print(reviews_df.head())


Обработка: 100%|██████████| 12500/12500 [01:45<00:00, 118.29file/s]

     ID  rating                                               text label
0   589      10  I've Seen The Beginning Of The Muppet Movie, B...   pos
1  6451       9  If it had been made 2 years later it would hav...   pos
2  2750       8  Very good "Precoder" starring Dick Barthelmess...   pos
3  5746      10  A young man discovers that life is precious af...   pos
4  2658      10  I'm always surprised, given that the famous ti...   pos


In [8]:
import os
import pandas as pd
from tqdm import tqdm

directory_path = '/kaggle/input/avaliacoes-de-filmes-dataset-imdb/aclImdb/test/neg'

data = []

for filename in tqdm(os.listdir(directory_path), desc="Обработка", unit="file"):
    if filename.endswith('.txt'):
        id_str, rating_str = filename[:-4].split('_')
        review_id = int(id_str)
        rating = int(rating_str)

        file_path = os.path.join(directory_path, filename)

        with open(file_path, 'r', encoding='utf-8') as file:
            review_text = file.read().strip()

        data.append({
            'ID': review_id,
            'rating': rating,
            'text': review_text,
            'label': 'neg' 
        })

neg_reviews_df = pd.DataFrame(data)
print(neg_reviews_df.head())


Обработка: 100%|██████████| 12500/12500 [01:46<00:00, 117.30file/s]

     ID  rating                                               text label
0   565       2  Committed doom and gloomer Peter Watkins goes ...   neg
1  6496       4  Most critics have written devastating about th...   neg
2  4408       1  Did I waste my time. This is very pretentious ...   neg
3  7628       1  What a stinker!!! I swear this movie was writt...   neg
4  6812       1  Ever had one of those nights when you couldn't...   neg


In [9]:
import pandas as pd

combined_df = pd.concat([reviews_df, neg_reviews_df], axis=0, ignore_index=True)

print(combined_df.head())
print(f"Шейп объединенного тренировочного датасета: {combined_df.shape}")


     ID  rating                                               text label
0   589      10  I've Seen The Beginning Of The Muppet Movie, B...   pos
1  6451       9  If it had been made 2 years later it would hav...   pos
2  2750       8  Very good "Precoder" starring Dick Barthelmess...   pos
3  5746      10  A young man discovers that life is precious af...   pos
4  2658      10  I'm always surprised, given that the famous ti...   pos
Шейп объединенного тренировочного датасета: (25000, 4)


In [10]:
import pandas as pd

combined_df['id'] = combined_df.index
duplicates = combined_df[combined_df.duplicated(subset='text', keep=False)]
sorted_duplicates = duplicates.sort_values(by='text')

print(sorted_duplicates)


          ID  rating                                               text label  \
16915   2183       2  "Go Fish" garnered Rose Troche rightly or wron...   neg   
12871  10977       2  "Go Fish" garnered Rose Troche rightly or wron...   neg   
18640   6089       3  "Three" is a seriously dumb shipwreck movie. M...   neg   
23145   4129       3  "Three" is a seriously dumb shipwreck movie. M...   neg   
20446   3124       4  (Spoilers)<br /><br />Oh sure it's based on Mo...   neg   
...      ...     ...                                                ...   ...   
17077   4125       1  this is the worst film I've seen in a long lon...   neg   
16367   1966       1  this movie sucks. did anyone notice that the e...   neg   
13630   3374       1  this movie sucks. did anyone notice that the e...   neg   
8715    2975      10  when I first heard about this movie, I noticed...   pos   
2090    2971      10  when I first heard about this movie, I noticed...   pos   

          id  
16915  16915

In [11]:
df_unique = combined_df.drop_duplicates(subset='text')
print(df_unique)

          ID  rating                                               text label  \
0        589      10  I've Seen The Beginning Of The Muppet Movie, B...   pos   
1       6451       9  If it had been made 2 years later it would hav...   pos   
2       2750       8  Very good "Precoder" starring Dick Barthelmess...   pos   
3       5746      10  A young man discovers that life is precious af...   pos   
4       2658      10  I'm always surprised, given that the famous ti...   pos   
...      ...     ...                                                ...   ...   
24995   6301       4  This is one of those inoffensive and mildly en...   neg   
24996     23       4  When people say children are annoying u think ...   neg   
24997   5134       3  OK, I don't want to upset anyone who enjoyed t...   neg   
24998   6660       2  Words can scarcely describe this movie. Loaded...   neg   
24999  12186       1  I watched this movie last night, i'm a huge fa...   neg   

          id  
0          0

In [12]:
df_unique.to_csv('/kaggle/working/combined_test.csv', encoding='utf-8')

Сделаем один общий датасет

In [13]:
df_1 = pd.read_csv('/kaggle/working/combined_train.csv')
df_2 = pd.read_csv('/kaggle/working/combined_test.csv')

In [14]:
combined_df_full = pd.concat([df_1, df_2], axis=0, ignore_index=True)


In [15]:
combined_df_full

,Unnamed: 0,ID,rating,text,label,id
0,0,2714,10,This was one of those wonderful rare moments i...,pos,0
1,1,589,10,Have you seen The Graduate? It was hailed as t...,pos,1
2,2,2211,8,"I don't watch a lot of TV, except for The Offi...",pos,2
3,3,2658,10,Kubrick again puts on display his stunning abi...,pos,3
4,4,8929,8,"First of all, I liked very much the central id...",pos,4
...,...,...,...,...,...,...
49700,24995,6301,4,This is one of those inoffensive and mildly en...,neg,24995
49701,24996,23,4,When people say children are annoying u think ...,neg,24996
49702,24997,5134,3,"OK, I don't want to upset anyone who enjoyed t...",neg,24997
49703,24998,6660,2,Words can scarcely describe this movie. Loaded...,neg,24998


Проверим дубликаты

In [16]:
import pandas as pd

combined_df_full['id'] = combined_df_full.index
duplicates = combined_df_full[combined_df_full.duplicated(subset='text', keep=False)]
sorted_duplicates = duplicates.sort_values(by='text')

print(sorted_duplicates)

       Unnamed: 0     ID  rating  \
48067       23329     69       4   
23105       23182  12298       4   
20777       20833   8091       4   
45265       20482   9507       4   
17948       17987   2534       3   
...           ...    ...     ...   
47781       23039   3886       3   
7726         7737   2405      10   
31715        6826   9865      10   
40697       15867   1537       2   
22945       23019  10203       2   

                                                    text label     id  
48067  "Witchery" might just be the most incoherent a...   neg  48067  
23105  "Witchery" might just be the most incoherent a...   neg  23105  
20777  (This is a review of the later English release...   neg  20777  
45265  (This is a review of the later English release...   neg  45265  
17948  * Some spoilers *<br /><br />This movie is som...   neg  17948  
...                                                  ...   ...    ...  
47781  With Knightly and O'Tool as the leads, this fi...   neg 

Дропнем дубликаты

In [17]:
df_unique = combined_df_full.drop_duplicates(subset='text')
print(df_unique)

       Unnamed: 0     ID  rating  \
0               0   2714      10   
1               1    589      10   
2               2   2211       8   
3               3   2658      10   
4               4   8929       8   
...           ...    ...     ...   
49700       24995   6301       4   
49701       24996     23       4   
49702       24997   5134       3   
49703       24998   6660       2   
49704       24999  12186       1   

                                                    text label     id  
0      This was one of those wonderful rare moments i...   pos      0  
1      Have you seen The Graduate? It was hailed as t...   pos      1  
2      I don't watch a lot of TV, except for The Offi...   pos      2  
3      Kubrick again puts on display his stunning abi...   pos      3  
4      First of all, I liked very much the central id...   pos      4  
...                                                  ...   ...    ...  
49700  This is one of those inoffensive and mildly en...   neg 

Итого у нас 49582 строки

Посмотрим, как там с распределением классов

In [18]:
import pandas as pd

label_counts = df_unique['label'].value_counts()

print(label_counts)


label
pos    24884
neg    24698
Name: count, dtype: int64


Как было поровну, так и осталось (в пределах погрешности)

Теперь посмотрим, сколько токенов у нас

In [19]:
import pandas as pd
from transformers import BertTokenizer
from tqdm import tqdm

df = df_unique

#возьмем токенизатор от берта
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tqdm.pandas(desc="Токенизация")

def count_tokens(text):
    tokens = tokenizer.tokenize(text)
    return len(tokens)

df['token_count'] = df['text'].progress_apply(count_tokens)

# Calculate stats
max_tokens = df['token_count'].max()
min_tokens = df['token_count'].min()
average_tokens = df['token_count'].mean()
total_tokens = df['token_count'].sum()

print(f"Максимальное кол-во токенов в отзыве: {max_tokens}")
print(f"Минимальное кол-во токенов в отзыве: {min_tokens}")
print(f"Среднее кол-во токенов в отзыве: {average_tokens:.2f}")
print(f"Всего токенов в датасете: {total_tokens}")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Токенизация: 100%|██████████| 49582/49582 [06:52<00:00, 120.27it/s]

Максимальное кол-во токенов в отзыве: 3155
Минимальное кол-во токенов в отзыве: 8
Среднее кол-во токенов в отзыве: 308.57
Всего токенов в датасете: 15299488



/tmp/ipykernel_30/3633013716.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['token_count'] = df['text'].progress_apply(count_tokens)


Посмотрим, сколько уникальных значений рейтингов (согласно описанию от создателей датасета, должны быть значения 1, 2, 3, 4, 7, 8, 9, 10 (без 5 и 6).

In [20]:
unique_ratings = df_unique['rating'].unique()
sorted_unique_ratings = sorted(unique_ratings)

print(sorted_unique_ratings)


[1, 2, 3, 4, 7, 8, 9, 10]


Все верно.

Теперь разделим нас огромный датасет на обучающую и тестовую выборки

Примечание: авторы датасета предлагают 25000 текстов на трейн и 25000 текстов на тест. Кажется, что лучше будет пойти по классическому пути разделения выборки на кусочки 0.8 к 0.2, чтобы было больше данных для обучения.

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = df_unique
print(df.head())

# Разделим в соотношении 0.8 на трейн к 0.2 на тест
train_df, test_df = train_test_split(
    df, 
    test_size=0.2, 
    stratify=df['label'], 
    random_state=42
)

print('Распределение классов для трейна:')
print(train_df['label'].value_counts(normalize=True))

print('Распределение классов для теста:')
print(test_df['label'].value_counts(normalize=True))

# Сохраним
train_df.to_csv('/kaggle/working/train_reviews.csv', index=False)
test_df.to_csv('/kaggle/working/test_reviews.csv', index=False)


   Unnamed: 0    ID  rating  \
0           0  2714      10   
1           1   589      10   
2           2  2211       8   
3           3  2658      10   
4           4  8929       8   

                                                text label  id  token_count  
0  This was one of those wonderful rare moments i...   pos   0          208  
1  Have you seen The Graduate? It was hailed as t...   pos   1          933  
2  I don't watch a lot of TV, except for The Offi...   pos   2          354  
3  Kubrick again puts on display his stunning abi...   pos   3          163  
4  First of all, I liked very much the central id...   pos   4          227  
Распределение классов для трейна:
label
pos    0.501878
neg    0.498122
Name: proportion, dtype: float64
Распределение классов для теста:
label
pos    0.501865
neg    0.498135
Name: proportion, dtype: float64


Распределение классов - примерно по 50 процентов для каждой выборки, как и должно быть.

Обучение классификаторов

Было решено обучить две модели: одна для предсказания сентимента отзыва, вторая - для предсказания класса.

Для обоих классификаторов было решено выбрать модель distilroberta-base. Модели roberta отлично улавливают контекст и информацию в текстовых данных. У нас отзывы к фильмам, где может быть очень важно улавливать контекстную информацию, потому что отзывы могут быть очень разнообразными в плане формального представления, что не всегда сможет уловить чисто формальная модель. Как пример, два отзыва на английском:

1) This movie is shit. (= фильм очень плохой, отзыв негативный)

2) This movie is the real shit. (= фильм очень хороший, отзыв позитивный)

Трансформенные модели, вероятно, смогли бы уловить разницу.

Также, была выбрана именно дистиллированная роберта, так как она меньше по размеру, что позволит ее развернуть на нашем сервисе с ограничением ресурсов для бесплатного использования.

В процессе обучения также применялся чанкинг, так как некоторые тексты значительно длиннее 512 токенов, которые может принять роберта на обучение в одном примере. 

Код для обучения модели для предсказания рейтингов отзывов по тексту:

In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# подгружаем наши уже разделенные на трейн и тест датасеты
train_df = pd.read_csv('/kaggle/working/train_reviews.csv')
test_df = pd.read_csv('/kaggle/working/test_reviews.csv')

# подгрузим токенизатор дистиллированной роберты
tokenizer = RobertaTokenizer.from_pretrained('distilbert/distilroberta-base')

#функция для чанкинга с максимальной длиной чанка 512 и страйдом 256
class SentimentChunkedDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, stride):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.stride = stride
        self.samples = self.create_samples(dataframe)

    def create_samples(self, dataframe):
        samples = []
        for idx in range(len(dataframe)):
            text = dataframe.iloc[idx, dataframe.columns.get_loc('text')]
            rating = dataframe.iloc[idx, dataframe.columns.get_loc('rating')]
            label = {1: 0, 2: 1, 3: 2, 4: 3, 7: 4, 8: 5, 9: 6, 10: 7}[rating]
            encodings = self.tokenizer(text, truncation=False, return_tensors="pt")
            token_count = encodings.input_ids.size(1)
            
            if token_count > self.max_len:
                for start in range(0, token_count, self.stride):
                    end = min(start + self.max_len, token_count)
                    chunk_encoding = {key: val[:, start:end] for key, val in encodings.items()}
                    chunk_encoding['labels'] = torch.tensor(label)
                    samples.append(chunk_encoding)
            else:
                encodings = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_len, return_tensors="pt")
                encodings['labels'] = torch.tensor(label)
                samples.append(encodings)
        return samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return {key: val.squeeze(0) for key, val in self.samples[idx].items()}

max_len = 512
stride = max_len // 2

#подготавливаем датасеты - на трейн идет весь трейн, а на валидацию весь тест
train_dataset = SentimentChunkedDataset(train_df, tokenizer, max_len, stride)
val_dataset = SentimentChunkedDataset(test_df, tokenizer, max_len, stride)

# Будем смотреть на accuracy, precision, recall и f1 во время обучения
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}

# подгружаем модель и обозначаем кол-во классов (рейтингов) - 8
model = RobertaForSequenceClassification.from_pretrained('distilbert/distilroberta-base', num_labels=8)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# обозначим гиперпараметры для обучения. предварительно ставим 15 эпох, чтобы смотреть за динамикой модели.
# сохраняем только одну лучшую по accuracy модель, чтобы экономить место и в любой момент прервать обучение
training_args = TrainingArguments(
    output_dir='.',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,
    metric_for_best_model='accuracy',
    load_best_model_at_end=True,
    num_train_epochs=15,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='.',
    logging_steps=10,
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset, 
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()


trainer.save_model('.')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1403 > 512). Running this sequence through the model will result in indexing errors


model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.270000,1.250743,0.507428,0.501359,0.439126,0.433963
2,1.161000,1.226578,0.508253,0.494072,0.445218,0.440419
3,1.077200,1.285691,0.500000,0.492454,0.456228,0.451379
4,0.852900,1.343958,0.508779,0.470395,0.464953,0.462921


KeyboardInterrupt: 

Обучение было прервано на 4 эпохе, так как уже на 3 начинается переобучение, и качество не растет значительно. В результате других попыток с разным кол-вом данных был такой же результат.

Сохраним модель в наш репозиторий на HuggingFace, чтобы пользоваться позже.

In [2]:
#входим в хаггинг фейс
from huggingface_hub import login
from huggingface_hub import HfApi, HfFolder, Repository
import os

hf_api_token = os.getenv('') 
login(token=hf_api_token)

In [3]:
# Сохраняем. Модель доступна в указанном репозитории
repo_name = "Gnider/roberta_dist_rat"

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Gnider/roberta_dist_rat/commit/9949d2f585a98696e7b738fe81e4e72b9f717844', commit_message='Upload tokenizer', commit_description='', oid='9949d2f585a98696e7b738fe81e4e72b9f717844', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Gnider/roberta_dist_rat', endpoint='https://huggingface.co', repo_type='model', repo_id='Gnider/roberta_dist_rat'), pr_revision=None, pr_num=None)

Как можно заметить, по метрике accuracy удалось достичь около 50% до того, как началось переобучение. Это немного, и еле затрагивает порог полезности (если вообще затрагивает). Кажется, что есть смысл провести эксперимент на точность в топ-n предсказаниях (например, какова будет точность, если одно из двух предсказаний истинно?), ведь оценка фильма человеком - очень субъективная вещь. Допустим,человек, дающий очень плохой отзыв, мог поставить 1 или 2, и обе эти оценки - показатели очень плохого фильма. Так же и с очень хорошим фильмом. Эксперименты будут ниже после обучения модели для сентимента.

Обучение модели для классификации сентимента

В сущности код такой же, как и для рейтингов, за исключением того, что теперь у нас два класса - позитив и негатив.

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

train_df = pd.read_csv('/kaggle/working/train_reviews.csv')
test_df = pd.read_csv('/kaggle/working/test_reviews.csv')

tokenizer = RobertaTokenizer.from_pretrained('distilbert/distilroberta-base')

class SentimentChunkedDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, stride):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.stride = stride
        self.samples = self.create_samples(dataframe)

    def create_samples(self, dataframe):
        samples = []
        for idx in range(len(dataframe)):
            text = dataframe.iloc[idx, dataframe.columns.get_loc('text')]
            label_text = dataframe.iloc[idx, dataframe.columns.get_loc('label')]
            label = 0 if label_text == 'neg' else 1
            encodings = self.tokenizer(text, truncation=False, return_tensors="pt")
            token_count = encodings.input_ids.size(1)
            
            if token_count > self.max_len:
                for start in range(0, token_count, self.stride):
                    end = min(start + self.max_len, token_count)
                    chunk_encoding = {key: val[:, start:end] for key, val in encodings.items()}
                    chunk_encoding['labels'] = torch.tensor(label)
                    samples.append(chunk_encoding)
            else:
                encodings = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_len, return_tensors="pt")
                encodings['labels'] = torch.tensor(label)
                samples.append(encodings)
        return samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return {key: val.squeeze(0) for key, val in self.samples[idx].items()}

max_len = 512
stride = max_len // 2

train_dataset = SentimentChunkedDataset(train_df, tokenizer, max_len, stride)
val_dataset = SentimentChunkedDataset(test_df, tokenizer, max_len, stride)

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}

model = RobertaForSequenceClassification.from_pretrained('distilbert/distilroberta-base', num_labels=2)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir='.',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,
    metric_for_best_model='accuracy',
    load_best_model_at_end=True,
    num_train_epochs=4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='.',
    logging_steps=10,
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset, 
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.save_model('.')


Так же сохраним на HuggingFace

In [ ]:
#входим в хаггинг фейс
from huggingface_hub import login
from huggingface_hub import HfApi, HfFolder, Repository
import os

hf_api_token = os.getenv('') 
login(token=hf_api_token)

In [ ]:
# Сохраняем. Модель доступна в указанном репозитории
repo_name = "Gnider/roberta_dist_sent"

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

К сожалению, лог обучения модели для сентимента не сохранился. Но это не беда, потому что мы можем подгрузить нашу модель с хаггинг фейса, куда мы ее сохранили после обучения, и проверить на тестовом датасете, что мы и сделаем в следующем разделе - оценка моделей.

Оценка моделей

Проверим предсказания для модели рейтингов:

In [30]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score
import numpy as np
from tqdm import tqdm

# Подгружаем наш тестовый датасет (который, кстати, и использовался во время валидации в обучении, но здесь мы делаем это отдельно)
test_df = pd.read_csv('/kaggle/working/test_reviews.csv')

# подгружаем токенизатор и модель с нашей репы в Хаггинг Фейс
model_name = 'Gnider/roberta_dist_rat'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

# Check device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Data preparation
class TestDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx]['text']
        label = self.dataframe.iloc[idx]['rating']
        encodings = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_len, return_tensors="pt")
        encodings['labels'] = torch.tensor(label, dtype=torch.long)
        return {key: val.squeeze(0) for key, val in encodings.items()}

max_len = 512
test_dataset = TestDataset(test_df, tokenizer, max_len)
test_loader = DataLoader(test_dataset, batch_size=32)

def eval_model(test_loader, model, top_k=1):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels']

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            probabilities = torch.softmax(outputs.logits, dim=-1)
            top_probs, top_preds = probabilities.topk(top_k, dim=-1)

            all_probs.extend(top_probs.cpu().numpy())
            all_preds.extend(top_preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    correct_count = 0
    for preds, label in zip(all_preds, all_labels):
        if label in preds:
            correct_count += 1

    accuracy = correct_count / len(all_labels)
    return accuracy, all_preds, all_probs, all_labels

# Проведем 3 эксперимента:
#точность топ-1 (один предсказанный рейтинг) - по идее должно быть около 0.5, как в обучении
top_1_accuracy, top_1_preds, top_1_probs, actual_labels = eval_model(test_loader, model, top_k=1)
# точность топ-2 - точность по двум наиболее вероятным предсказанным рейтингами (если один из двух попадает - то это засчитывается за верное предсказание)
top_2_accuracy, top_2_preds, top_2_probs, _ = eval_model(test_loader, model, top_k=2)
# точность топ-3 - то же самое, только с топ-3
top_3_accuracy, top_3_preds, top_3_probs, _ = eval_model(test_loader, model, top_k=3)



Evaluating: 100%|██████████| 310/310 [01:47<00:00,  2.88it/s]


In [31]:
# Сделаем несколько примеров для наглядности
num_examples = 5
example_df = pd.DataFrame({
    'text': test_df['text'][:num_examples],
    'actual_label': actual_labels[:num_examples],
    'top_1_prediction': [list(preds) for preds in top_1_preds[:num_examples]],
    'top_1_probabilities': [list(probs) for probs in top_1_probs[:num_examples]],
    'top_2_prediction': [list(preds) for preds in top_2_preds[:num_examples]],
    'top_2_probabilities': [list(probs) for probs in top_2_probs[:num_examples]],
    'top_3_prediction': [list(preds) for preds in top_3_preds[:num_examples]],
    'top_3_probabilities': [list(probs) for probs in top_3_probs[:num_examples]],
})

print("\nТочность:")
print(f"Top-1: {top_1_accuracy:.2%}")
print(f"Top-2: {top_2_accuracy:.2%}")
print(f"Top-3: {top_3_accuracy:.2%}")


Точность:
Top-1: 4.30%
Top-2: 28.63%
Top-3: 39.88%


In [32]:
example_df

,text,actual_label,top_1_prediction,top_1_probabilities,top_2_prediction,top_2_probabilities,top_3_prediction,top_3_probabilities
0,What a gargantuan pile of malodorous ordure! Y...,1,[0],[0.9810503],"[0, 1]","[0.9810503, 0.016247742]","[0, 1, 2]","[0.9810503, 0.016247742, 0.0015734542]"
1,"I was first introduced to ""Eddie"" by friends f...",10,[7],[0.859163],"[7, 6]","[0.859163, 0.0935864]","[7, 6, 5]","[0.859163, 0.0935864, 0.04039549]"
2,"Lexi befriends Jennifer, a thin, intelligent g...",10,[4],[0.32465562],"[4, 5]","[0.32465562, 0.25848362]","[4, 5, 3]","[0.32465562, 0.25848362, 0.12228855]"
3,the real plot...<br /><br />A group of post-Ci...,1,[0],[0.82128704],"[0, 1]","[0.82128704, 0.12733063]","[0, 1, 2]","[0.82128704, 0.12733063, 0.03112289]"
4,I actually like Asylum movies. I've made it a ...,1,[0],[0.96615577],"[0, 1]","[0.96615577, 0.028151466]","[0, 1, 2]","[0.96615577, 0.028151466, 0.0037979262]"


Вопреки нашим ожиданиям, даже несмотря на то что во время обучения метрика accuracy показывала 50%, здесь результаты значительно хуже. Даже топ-3 не дотягивают до 50%. Вероятно, модель сильно переобучилась. Необходимы дальнейшие эксперименты.  

Ввиду ограниченности вермени мы не можем провести дальнейшие опыты с обучением, поэтому в сервис вложим эту модель. Благо, всегда можно обучить новую улучшеную модель и легко всунуть в код сервиса.

Теперь проверим сентимент.

In [33]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score
from tqdm import tqdm

test_df = pd.read_csv('/kaggle/working/test_reviews.csv')

model_name = 'Gnider/roberta_dist_sent'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

class SentimentTestDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.max_len = max_len
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx]['text']
        label_text = self.dataframe.iloc[idx]['label']
        label = 0 if label_text == 'neg' else 1
        encodings = self.tokenizer(
            text, truncation=True, padding='max_length', 
            max_length=self.max_len, return_tensors="pt"
        )
        encodings['labels'] = torch.tensor(label, dtype=torch.long)
        return {key: val.squeeze(0) for key, val in encodings.items()}

max_len = 512
test_dataset = SentimentTestDataset(test_df, tokenizer, max_len)
test_loader = DataLoader(test_dataset, batch_size=32)

def evaluate_model(loader, model):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Оценка"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy, all_labels, all_preds

#общая оценка точности предсказания сентимента
accuracy, actual_labels, predicted_labels = evaluate_model(test_loader, model)
print(f"Accuracy: {accuracy:.2%}")

# 50 примеров
example_count = 50
examples_df = pd.DataFrame({
    'text': test_df['text'][:example_count],
    'actual_label': ['pos' if label == 1 else 'neg' for label in actual_labels[:example_count]],
    'predicted_label': ['pos' if label == 1 else 'neg' for label in predicted_labels[:example_count]],
})

examples_df


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/751 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

Оценка: 100%|██████████| 310/310 [01:50<00:00,  2.79it/s]

Accuracy: 98.60%


,text,actual_label,predicted_label
0,What a gargantuan pile of malodorous ordure! Y...,neg,neg
1,"I was first introduced to ""Eddie"" by friends f...",pos,pos
2,"Lexi befriends Jennifer, a thin, intelligent g...",pos,pos
3,the real plot...<br /><br />A group of post-Ci...,neg,neg
4,I actually like Asylum movies. I've made it a ...,neg,neg
5,I really liked this movie. If other people wan...,pos,pos
6,**** Possible Spoiler **** <br /><br />If you ...,neg,neg
7,"Oh man, I know what your thinking: ""With a tit...",neg,neg
8,Never viewed this film and enjoyed the singing...,pos,pos
9,Maiden Voyage is just that. I'd like to say st...,neg,neg


Здесь все значительно лучше: точность предсказания сентимента - 98.6%. Из примеров все 50 сентиментов предсказаны верно.


Следующий шаг - оформление сервиса Django и развертывание. Подробнее об этом - в файле отчета.